## Dress review text mining
Using bag of words and Naïve Bayes

<b>Bag of words</b> is an algorithm that is used for text data. It takes all words from a text document seperatly and extends this over the columns. In the rows, the text is shown and will have a 1 as a count when the words in the column appears in the sentence.

<b>Naïve Bayes</b> is a propability algorithm that is used for classification problems. By learning from previous text , which is by bag of words converted to a document feature matrix, it can predict a certain Y value. For instance, this can be the person who said the sentence or the mood someone is in. What makes it naïve is that it assumes all specific words are independent, but they are not. Because they always rely on each other and in a full sentence can have a different actual meaning.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer #This creates a dictionary from a series of text.
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB

In [69]:
complete_df = pd.read_csv('clothing.csv') #The whole set
df = complete_df.loc[(complete_df['Class Name'] == 'Dresses')] #Just the dresses
df.head(5)
# (len(df)) #we still have 6319 reviews.

Unnamed: 0  Clothing ID  Age                    Title  \
1           1         1080   34                      NaN   
2           2         1077   60  Some major design flaws   
5           5         1080   49  Not for the very petite   
8           8         1077   24               Flattering   
9           9         1077   34        Such a fun dress!   

                                         Review Text  Rating  Recommended IND  \
1  Love this dress!  it's sooo pretty.  i happene...       5                1   
2  I had such high hopes for this dress and reall...       3                0   
5  I love tracy reese dresses, but this one is no...       2                0   
8  I love this dress. i usually get an xs but it ...       5                1   
9  I'm 5"5' and 125 lbs. i ordered the s petite t...       5                1   

   Positive Feedback Count Division Name Department Name Class Name  
1                        4       General         Dresses    Dresses  
2                        0       General         Dresses    Dresses  
5                        4       General         Dresses    Dresses  
8                        0       General         Dresses    Dresses  
9                        0       General         Dresses    Dresses

Here we determine what column to take for the text analysis. The stop words are going to be taken out because they're not going to help us any further.

In [81]:
text = df['Review Text'].values.astype('U') #Taking the text from the df. We need to convert it to Unicode
vect = CountVectorizer(stop_words='english') #Words that are not relevant
vect = vect.fit(text) #We fit the model with the words from the review text
docu_feat = vect.transform(text) #For the matrix
df.head()

Unnamed: 0  Clothing ID  Age                    Title  \
1           1         1080   34                      NaN   
2           2         1077   60  Some major design flaws   
5           5         1080   49  Not for the very petite   
8           8         1077   24               Flattering   
9           9         1077   34        Such a fun dress!   

                                         Review Text  Rating  Recommended IND  \
1  Love this dress!  it's sooo pretty.  i happene...       5                1   
2  I had such high hopes for this dress and reall...       3                0   
5  I love tracy reese dresses, but this one is no...       2                0   
8  I love this dress. i usually get an xs but it ...       5                1   
9  I'm 5"5' and 125 lbs. i ordered the s petite t...       5                1   

   Positive Feedback Count Division Name Department Name Class Name   Big2  
1                        4       General         Dresses    Dresses   True  
2                        0       General         Dresses    Dresses  False  
5                        4       General         Dresses    Dresses  False  
8                        0       General         Dresses    Dresses   True  
9                        0       General         Dresses    Dresses   True

In [70]:
feature_names = vect.get_feature_names() #Get all seperate words from the reviews
print(f"There are {len(feature_names)} words in the vocabulary. A selection: {feature_names[3000:3100]}") #Some words

There are 8080 words in the vocabulary. A selection: ['floreat', 'florida', 'flory', 'flounce', 'flounces', 'flouncy', 'flow', 'flowed', 'flower', 'flowers', 'flowery', 'flowey', 'flowier', 'flowing', 'flowly', 'flown', 'flows', 'flowy', 'flub', 'fluctuate', 'fluctuates', 'fluctuating', 'fluctuations', 'fluevogs', 'fluff', 'fluffs', 'fluffy', 'fluid', 'fluidity', 'fluke', 'flung', 'fluorescent', 'flushed', 'flutter', 'fluttery', 'fly', 'flyaway', 'flying', 'fo', 'foam', 'focus', 'focused', 'fol', 'fold', 'folded', 'folding', 'folds', 'folk', 'folks', 'follow', 'followed', 'following', 'follows', 'fond', 'fondly', 'food', 'fool', 'fooled', 'foot', 'football', 'footless', 'footwear', 'force', 'forces', 'forcing', 'forearm', 'foresee', 'forest', 'forever', 'forewarned', 'forget', 'forgettin', 'forgive', 'forgiven', 'forgiving', 'forgot', 'forgotten', 'forh', 'form', 'formal', 'formally', 'formfitting', 'formless', 'forth', 'fortunate', 'fortunately', 'fortune', 'fortuny', 'forwar', 'forwa

Now all words are seperated, we can start counting them. The first number before the comma stands for the row (the review) and the number behind the comma stands for the column (the word). This means that the review on row 21 has words in column 4 and 45.

In [40]:
docu_feat = vect.transform(text) #Makes the matrix
print(docu_feat[0:50,0:50])

  (2, 8)	1
  (20, 38)	1
  (21, 4)	1
  (21, 45)	1
  (22, 12)	1
  (26, 40)	1
  (35, 12)	2
  (39, 31)	1


In [41]:
df_words = pd.concat([df, pd.DataFrame(docu_feat.toarray()).reset_index()], axis=1)
df_words.head(5)

Unnamed: 0  Clothing ID   Age                    Title  \
0         NaN          NaN   NaN                      NaN   
1         1.0       1080.0  34.0                      NaN   
2         2.0       1077.0  60.0  Some major design flaws   
3         NaN          NaN   NaN                      NaN   
4         NaN          NaN   NaN                      NaN   

                                         Review Text  Rating  Recommended IND  \
0                                                NaN     NaN              NaN   
1  Love this dress!  it's sooo pretty.  i happene...     5.0              1.0   
2  I had such high hopes for this dress and reall...     3.0              0.0   
3                                                NaN     NaN              NaN   
4                                                NaN     NaN              NaN   

   Positive Feedback Count Division Name Department Name  ... 8070  8071  \
0                      NaN           NaN             NaN  ...  0.0   0.0   
1                      4.0       General         Dresses  ...  0.0   0.0   
2                      0.0       General         Dresses  ...  0.0   0.0   
3                      NaN           NaN             NaN  ...  0.0   0.0   
4                      NaN           NaN             NaN  ...  0.0   0.0   

   8072  8073  8074  8075  8076  8077  8078  8079  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 8092 columns]

This shows the whole document feature matrix. All reviews are in the rows and words in the columns. It mostly contains zero's and some 1's.

In [82]:
nb = MultinomialNB() #create the model
X = docu_feat #That's gonna be all the X's to predict the Y
y = df['Rating'] #The Y to predict

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) #split the data and store it

nb = nb.fit(X_train, y_train) #fit the model X=features, y=character

In [72]:
y_test_p = nb.predict(X_test)
nb.score(X_test, y_test)

0.5954641350210971

The model has an accuracy of 59.54%. Since there are 5 possibilities (A rating of 1,2,3,4 or 5), this score is pretty good.

In [73]:
df['Rating'].value_counts(normalize=True)

5    0.537585
4    0.220763
3    0.132616
2    0.072955
1    0.036082
Name: Rating, dtype: float64

In [74]:
cm = confusion_matrix(y_test, y_test_p)
cm = pd.DataFrame(cm, index=['1', '2', '3', '4', '5' ], columns=['1P', '2P', '3P', '4P', '5P'])
cm

1P  2P  3P   4P   5P
1   2  10  24   10   19
2   2   7  47   42   45
3   1   5  78   86   71
4   0   1  30  131  264
5   1   1  10   98  911

This shows how many of the actual ratings are predicted right. The model works the best for the 5 rating, probably because it has the most data as input for the review with this rating. We also see that in the classification report.

In [75]:
nb.classes_

array([1, 2, 3, 4, 5])

In [76]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_p, nb.classes_)) #Receives the column names from nb.classes_

              precision    recall  f1-score   support

           1       0.33      0.03      0.06        65
           2       0.29      0.05      0.08       143
           3       0.41      0.32      0.36       241
           4       0.36      0.31      0.33       426
           5       0.70      0.89      0.78      1021

   micro avg       0.60      0.60      0.60      1896
   macro avg       0.42      0.32      0.32      1896
weighted avg       0.54      0.60      0.55      1896



### Predict propabilities

With 'predict_proba' the probability per review is going to be predicted.

In [77]:
print(df.iloc[3,4]) #The column where the review is in
print(nb.predict_proba(X[0]))

I love this dress. i usually get an xs but it runs a little snug in bust so i ordered up a size. very flattering and feminine with the usual retailer flair for style.
[[2.57257757e-14 2.93525618e-08 1.08330089e-05 2.00953248e-02
  9.79893813e-01]]


In [78]:
for i in range(5):
    prob = nb.predict_proba(X[i])
    print(f"Line: {i}. {df.iloc[i,4]}")
    print(f"Rate 1: {prob[0,0]}, Rate 2: {prob[0,1]}, Rate 3: {prob[0,2]}, Rate 4: {prob[0,3]}, Rate 5: {prob[0,4]}")

Line: 0. Love this dress!  it's sooo pretty.  i happened to find it in a store, and i'm glad i did bc i never would have ordered it online bc it's petite.  i bought a petite and am 5'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.
Rate 1: 2.5725775727949813e-14, Rate 2: 2.9352561841939775e-08, Rate 3: 1.0833008881625227e-05, Rate 4: 0.02009532482817664, Rate 5: 0.9798938128103605
Line: 1. I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c
Rate 1: 1.032477709762782

This is the prediction per rating. I didn't split it up to a negative or a positive review (>3).